In [ ]:
import torch
import torch.nn as nn
import joblib
import numpy as np
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from facenet_pytorch import InceptionResnetV1
from torchvision import transforms
from PIL import Image
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
import cv2

In [ ]:
# Initialize device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
# Preprocessing transformation for FaceNet
transform = transforms.Compose([
    transforms.Resize((160, 160)),  # Resize images to FaceNet's expected input size
    transforms.ToTensor(),         # Convert images to PyTorch tensors
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])  # Normalize images
])

In [ ]:
# Load training and test datasets
train_dataset = datasets.ImageFolder('/content/Dataset/Dataset/Dataset_cropped/train', transform=transform)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

test_dataset = datasets.ImageFolder('/content/Dataset/Dataset/Dataset_cropped/test', transform=transform)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [ ]:
# Class names
class_names = train_dataset.classes
print(f"Classes: {class_names}")

# Load pre-trained FaceNet model
facenet_model = InceptionResnetV1(pretrained='vggface2').eval().to(device)

In [ ]:
# Function to extract embeddings
def get_embeddings(data_loader, model):
    embeddings = []
    labels = []

    with torch.no_grad():
        for imgs, lbls in data_loader:
            imgs = imgs.to(device)
            emb = model(imgs).cpu().numpy()
            embeddings.append(emb)
            labels.append(lbls.numpy())

    return np.vstack(embeddings), np.hstack(labels)

In [ ]:
# Extract embeddings for training and test sets
train_embeddings, train_labels = get_embeddings(train_loader, facenet_model)
test_embeddings, test_labels = get_embeddings(test_loader, facenet_model)

print("Train embeddings shape:", train_embeddings.shape)
print("Test embeddings shape:", test_embeddings.shape)

In [ ]:
# Save embeddings and labels
np.save('train_embeddings.npy', train_embeddings)
np.save('train_labels.npy', train_labels)
np.save('test_embeddings.npy', test_embeddings)
np.save('test_labels.npy', test_labels)

In [ ]:
# Train the SVM on the FaceNet embeddings
svm_model = SVC(kernel='linear', probability=True)
svm_model.fit(train_embeddings, train_labels)

In [ ]:
# Save the trained SVM model
joblib.dump(svm_model, 'svm_model.pkl')

In [ ]:
# Evaluate on the test set
predictions = svm_model.predict(test_embeddings)
accuracy = accuracy_score(test_labels, predictions)
print(f"Test Accuracy: {accuracy * 100:.2f}%")

In [ ]:
# Define a simple classifier
class FaceClassifier(nn.Module):
    def __init__(self, embedding_dim, num_classes):
        super(FaceClassifier, self).__init__()
        self.fc = nn.Linear(embedding_dim, num_classes)

    def forward(self, x):
        return self.fc(x)

In [ ]:
# Create model and train
num_classes = len(class_names)
classifier_model = FaceClassifier(embedding_dim=512, num_classes=num_classes).to(device)

In [ ]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(classifier_model.parameters(), lr=0.001)

In [ ]:
# Training loop
for epoch in range(10):
    classifier_model.train()
    for embeddings, labels in zip(train_embeddings, train_labels):
        embeddings = torch.tensor(embeddings, dtype=torch.float32).to(device)
        labels = torch.tensor(labels, dtype=torch.long).to(device)

        outputs = classifier_model(embeddings)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

print("Training completed!")

In [ ]:
# Load the saved SVM model
svm_model = joblib.load('svm_model.pkl')

In [ ]:
# Function for face recognition using webcam feed
def recognize_face(image, facenet_model, classifier_model, class_names):
    # Detect face, preprocess, and get embedding
    embedding = facenet_model(image.unsqueeze(0).to(device)).cpu().detach().numpy()

    # Predict using the classifier
    prediction = classifier_model(torch.tensor(embedding, dtype=torch.float32).to(device))
    _, predicted_class = torch.max(prediction, dim=1)
    return class_names[predicted_class.item()]

In [ ]:
# Function to run face recognition on webcam feed
def run_face_recognition():
    cap = cv2.VideoCapture(0)

    while True:
        ret, frame = cap.read()

        if not ret:
            break

        # Convert frame to PIL image
        pil_image = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

        # Preprocess image
        image = preprocess(pil_image).unsqueeze(0).to(device)

        # Get face embeddings
        with torch.no_grad():
            embedding = facenet_model(image)

        # Recognize face using the trained classifier
        predicted_class_name = recognize_face(embedding, facenet_model, classifier_model, class_names)
        print(f"Predicted Class: {predicted_class_name}")

        # Draw bounding box around the face (simple for demonstration)
        # (In real-world applications, you'd want to use a face detection model like MTCNN)
        cv2.putText(frame, f'Face: {predicted_class_name}', (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

        # Display the resulting frame
        cv2.imshow('Face Recognition', frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()


In [ ]:
# Run face recognition on webcam
run_face_recognition()

In [ ]:
# Save the models
torch.save(facenet_model.state_dict(), 'facenet_model.pth')
torch.save(classifier_model.state_dict(), 'classifier_model.pth')

In [ ]:
# Save class names
with open('class_names.json', 'w') as f:
    json.dump(class_names, f)